# **Neural Network from Scratch**

# Deep Learning Lab Assignment
# MIT Academy of Engineering, Alandi, Pune
# Class: BTech - Deep Learning

**Name:-** Nabil Ansari \
**MSID:-** 75 \
**PRN:-** 202302040004


# Objective
This assignment involves implementing a simple feedforward neural network from scratch in Python. The focus is on understanding key neural network concepts such as forward pass, backpropagation, and gradient descent optimization.


In [3]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Activation Function

def sigmoid(x):
    """Sigmoid Activation Function: This function maps input values to the range (0,1), making it useful for neural networks dealing with probabilities."""
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """Derivative of Sigmoid Activation Function: This is used during backpropagation to update weights accordingly."""
    return x * (1 - x)

"""
# Neural Network Architecture
The neural network consists of:
- 4 input neurons (corresponding to the features in the Iris dataset)
- 2 hidden layers with 6 and 5 neurons respectively
- 3 output neurons (corresponding to the three classes in the dataset)
The network uses the Sigmoid activation function for both hidden and output layers.
"""

# Neural Network Class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        """Initialize network architecture with weights and biases."""
        self.input_size = input_size
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.output_size = output_size

        self.weights_input_hidden1 = np.random.randn(self.input_size, self.hidden_size1)
        self.bias_hidden1 = np.random.randn(1, self.hidden_size1)

        self.weights_hidden1_hidden2 = np.random.randn(self.hidden_size1, self.hidden_size2)
        self.bias_hidden2 = np.random.randn(1, self.hidden_size2)

        self.weights_hidden2_output = np.random.randn(self.hidden_size2, self.output_size)
        self.bias_output = np.random.randn(1, self.output_size)

    def forward(self, X):
        """Forward pass computation: Propagates input data through the network layers."""
        self.hidden1_input = np.dot(X, self.weights_input_hidden1) + self.bias_hidden1
        self.hidden1_output = sigmoid(self.hidden1_input)

        self.hidden2_input = np.dot(self.hidden1_output, self.weights_hidden1_hidden2) + self.bias_hidden2
        self.hidden2_output = sigmoid(self.hidden2_input)

        self.output_layer_input = np.dot(self.hidden2_output, self.weights_hidden2_output) + self.bias_output
        self.output_layer_output = sigmoid(self.output_layer_input)

        return self.output_layer_output

    def backward(self, X, y, learning_rate):
        """Backward pass computation: Computes errors and updates weights using gradient descent."""
        error_output = y - self.output_layer_output
        output_layer_delta = error_output * sigmoid_derivative(self.output_layer_output)

        error_hidden2 = output_layer_delta.dot(self.weights_hidden2_output.T)
        hidden2_layer_delta = error_hidden2 * sigmoid_derivative(self.hidden2_output)

        error_hidden1 = hidden2_layer_delta.dot(self.weights_hidden1_hidden2.T)
        hidden1_layer_delta = error_hidden1 * sigmoid_derivative(self.hidden1_output)

        self.weights_hidden2_output += self.hidden2_output.T.dot(output_layer_delta) * learning_rate
        self.bias_output += np.sum(output_layer_delta, axis=0, keepdims=True) * learning_rate

        self.weights_hidden1_hidden2 += self.hidden1_output.T.dot(hidden2_layer_delta) * learning_rate
        self.bias_hidden2 += np.sum(hidden2_layer_delta, axis=0, keepdims=True) * learning_rate

        self.weights_input_hidden1 += X.T.dot(hidden1_layer_delta) * learning_rate
        self.bias_hidden1 += np.sum(hidden1_layer_delta, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, epochs, learning_rate):
        """Train the network over multiple epochs using forward and backward propagation."""
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y, learning_rate)

            if epoch % 1000 == 0:
                loss = np.mean(np.square(y - self.output_layer_output))
                print(f"Epoch {epoch} - Loss: {loss}")

"""
# Dataset
The Iris dataset is used for classification. It consists of 150 samples with 4 features each, categorized into 3 different species.
"""

# Load Iris Dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse_output=False)
y_one_hot = encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_one_hot, test_size=0.2, random_state=42)

# Train the Neural Network
nn = NeuralNetwork(input_size=4, hidden_size1=6, hidden_size2=5, output_size=3)
nn.train(X_train, y_train, epochs=10000, learning_rate=0.1)

"""
# Model Evaluation
The trained model is evaluated on test data, and its accuracy is printed.
"""

# Model Evaluation
predictions = nn.forward(X_test)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

accuracy = np.mean(predicted_labels == true_labels)
print(f"\nModel Accuracy: {accuracy * 100:.2f}%")

"""
# User Input for Prediction
Allows the user to enter flower measurements and predicts the corresponding Iris species.
"""

def predict_species():
    print("\nEnter flower measurements:")
    sepal_length = float(input("Sepal Length (cm): "))
    sepal_width = float(input("Sepal Width (cm): "))
    petal_length = float(input("Petal Length (cm): "))
    petal_width = float(input("Petal Width (cm): "))

    user_input = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
    user_input_scaled = scaler.transform(user_input)

    prediction = nn.forward(user_input_scaled)
    predicted_class = np.argmax(prediction)

    species = ["Setosa", "Versicolor", "Virginica"]
    print(f"\nPredicted Species: {species[predicted_class]}")

predict_species()

Epoch 0 - Loss: 0.3218585740302536
Epoch 1000 - Loss: 0.006094780772675604
Epoch 2000 - Loss: 0.0056595512961892394
Epoch 3000 - Loss: 0.005592621914023347
Epoch 4000 - Loss: 0.005561622863923562
Epoch 5000 - Loss: 0.005539138759544439
Epoch 6000 - Loss: 0.005521314482994205
Epoch 7000 - Loss: 0.005507345993828759
Epoch 8000 - Loss: 0.0054960792959482035
Epoch 9000 - Loss: 0.005486459420232381

Model Accuracy: 100.00%

Enter flower measurements:
Sepal Length (cm): 5.1
Sepal Width (cm): 3.5
Petal Length (cm): 1.4
Petal Width (cm): 0.2

Predicted Species: Setosa


# Declaration:
I, Nabil Ansari, confirm that the work submitted in this assignment is my own and has been completed
following academic integrity guidelines. The code is uploaded on my GitHub repository account, and the
repository link is provided below:
# GitHub Repository Link: https://github.com/nabil-repo/DL
# Signature:Nabil Aman Aasif Ahmad Ansari